# Assignment 4 - Association Rules Mining
## Data Mining

Apriori Algorithm from Scratch

by Ardian - 2106638173

Information System - Faculty of Computer Science

Universitas Indonesia

---

## Libraries

In [1]:
import pandas as pd
from collections import defaultdict

## Input Format

Data transaksi diambil dari kasus soal, yaitu:

- Transaction 1: {Laptop, Mouse, Keyboard, Headphones}
- Transaction 2: {Laptop, Mouse, Keyboard}
- Transaction 3: {Mouse, Keyboard, Headphones}
- Transaction 4: {Laptop, Keyboard, Headphones}
- Transaction 5: {Laptop, Mouse, Keyboard, Webcam}

In [2]:
data = {
    'Laptop': [1, 1, 0, 1, 1],
    'Mouse': [1, 1, 1, 0, 1],
    'Keyboard': [1, 1, 1, 1, 1],
    'Headphones': [1, 0, 1, 1, 0],
    'Webcam': [0, 0, 0, 0, 1]
}

transactions_df = pd.DataFrame(data)
transactions_df.index += 1
transactions_df.index.name = 'TransactionID'

transactions_df

,Laptop,Mouse,Keyboard,Headphones,Webcam
TransactionID,,,,,
1,1,1,1,1,0
2,1,1,1,0,0
3,0,1,1,1,0
4,1,0,1,1,0
5,1,1,1,0,1


> Setiap row merepresentasikan satu transaksi. Kolom/item bernilai 1 apabila item tersebut ada pada row ransaksi tersebut

## Utility Functions

### `get_support()` function:

In [3]:
def get_support(transactions, itemset):
  match_count = 0
  for transaction in transactions:
    if itemset.issubset(transaction):
      match_count += 1

  return float(match_count/len(transactions))

### `self_join()` function:

In [4]:
def self_join(frequent_item_sets_per_level, level):
  current_level_candidates = list()
  last_level_items = frequent_item_sets_per_level[level - 1]

  if len(last_level_items) == 0:
    return current_level_candidates

  for i in range(len(last_level_items)):
    for j in range(i+1, len(last_level_items)):
      itemset_i = last_level_items[i][0]
      itemset_j = last_level_items[j][0]
      union_set = itemset_i.union(itemset_j)

      if union_set not in current_level_candidates and len(union_set) == level:
        current_level_candidates.append(union_set)

  return current_level_candidates

### `generate_itemsets()` function:

In [5]:
def generate_itemsets(item_set):
  item_sets = list()
  for item in item_set:
    temp = item_set.copy()
    temp.remove(item)
    item_sets.append(temp)

  return item_sets

### `is_itemset_frequent()` function:

In [6]:
def is_itemset_frequent(item_set, prev_level_sets):
  item_sets = generate_itemsets(item_set)

  for item_set in item_sets:
    if item_set not in prev_level_sets:
      return False
  return True

### `prune()` function:

In [7]:
def prune(frequent_item_sets_per_level, level, candidate_set):
  post_pruning_set = list()
  if len(candidate_set) == 0:
    return post_pruning_set

  prev_level_sets = list()
  for item_set, _ in frequent_item_sets_per_level[level - 1]:
    prev_level_sets.append(item_set)

  for item_set in candidate_set:
    if is_itemset_frequent(item_set, prev_level_sets):
        post_pruning_set.append(item_set)

  return post_pruning_set

## Apriori Algorithm

### Build the algorithm function `Apriori()`:

In [8]:
def Apriori(transactions_df, minsup):
  frequent_item_sets_per_level = defaultdict(list)

  # Extract item names and create a dictionary to map items to indices
  item_list = list(transactions_df.columns)
  item_dict = {item: i + 1 for i, item in enumerate(item_list)}

  # Process transactions dataframe to be processable
  transactions = []
  for _, row in transactions_df.iterrows():
    transaction = set()
    for item in item_dict:
      if row[item] == 1:
        transaction.add(item)
    transactions.append(transaction)

  # Generate frequent item sets for level 1
  for item in item_list:
    support = get_support(transactions, {item})
    if support >= minsup:
      frequent_item_sets_per_level[1].append(({item}, support))

  # Generate frequent item sets for higher levels
  for level in range(2, len(item_list) + 1):
    current_level_candidates = self_join(frequent_item_sets_per_level, level)
    post_pruning_candidates = prune(frequent_item_sets_per_level, level, current_level_candidates)
    if len(post_pruning_candidates) == 0:
      break

    for item_set in post_pruning_candidates:
      support = get_support(transactions, item_set)
      if support >= minsup:
        frequent_item_sets_per_level[level].append((item_set, support))

  return frequent_item_sets_per_level

### Test the algorithm function:

Berdasarkan jawaban saya dari soal nomor 1 dan 2, frequent itemsets yang valid dengan minimum support 0.6 adalah sebagai berikut:
1.	{Laptop} dengan support 0.8
2.	{Keyboard} dengan support 1
3.	{Mouse} dengan support 0.8
4.	{Headphones} dengan support 0.6
5.	{Laptop, Keyboard} dengan support 0.8
6.	{Laptop, Mouse} dengan support 0.6
7.	{Keyboard, Mouse} dengan support 0.8
8.	{Keyboard, Headphones} dengan support 0.6
9.	{Laptop, Keyboard, Mouse} dengan support 0.6

Jika hasil test algoritma sesuai dengan jawaban tersebut, maka dapat disimpulkan algoritma bekerja dengan baik. Mari kita coba:

In [12]:
minsup = 0.6

frequent_itemsets = Apriori(transactions_df, minsup)

for level, frequent_item_sets in frequent_itemsets.items():
  print(f"Level {level} frequent itemsets:")
  counter = 1
  for item_set, support in frequent_item_sets:
    items = ", ".join(item for item in item_set)
    print(f"{counter}. {{{items}}} with {support} support")
    counter += 1
  print()

Level 1 frequent itemsets:
1. {Laptop} with 0.8 support
2. {Mouse} with 0.8 support
3. {Keyboard} with 1.0 support
4. {Headphones} with 0.6 support

Level 2 frequent itemsets:
1. {Mouse, Laptop} with 0.6 support
2. {Laptop, Keyboard} with 0.8 support
3. {Mouse, Keyboard} with 0.8 support
4. {Keyboard, Headphones} with 0.6 support

Level 3 frequent itemsets:
1. {Mouse, Laptop, Keyboard} with 0.6 support



> SESUAI! Algoritma berjalan dengan baik!